In [21]:
import geopandas as gpd
import pandas as pd
import rasterio
import boto3
import os
import utm
#import pyproj

In [3]:
download_dir = '../downloads/crop_ver2/'
image_files = os.listdir(download_dir)

In [4]:
bucket_name = 'opera-calval-database-dswx'

In [22]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [23]:
#obj = s3.Object(bucket_name,'site.geojson')
#site_table_data = obj.get()['Body']
#sites = gpd.read_file(site_table_data)
sites = gpd.read_file('./new_chips (1).geojson')
sites

,site_name,water_pixels_orig,inundated_vegetation_orig,total_pixels_orig,water_pixels_crop,inundated_vegetation_crop,total_pixels_crop,percent_water_orig,percent_inundated_vegetation_orig,percent_water_crop,percent_inundated_vegetation_crop,percent_total_water_orig,percent_total_water_crop,geometry
0,3_38,0.0,0.0,1252157,0.0,0.0,250000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,"POLYGON ((-159.66522 69.26009, -159.66385 69.2..."
1,4_27,286582.0,8919.0,1251485,78861.0,3701.0,250000,22.899356,0.712673,31.5444,1.4804,23.612029,33.0248,"POLYGON ((-113.00031 67.04574, -112.99662 67.0..."
2,4_43,556111.0,41908.0,1244440,192790.0,2307.0,250000,44.687651,3.367619,77.1160,0.9228,48.055270,78.0388,"POLYGON ((17.81813 65.99817, 17.81319 65.95337..."
3,3_1,17647.0,246974.0,1244553,7271.0,41560.0,250000,1.417939,19.844394,2.9084,16.6240,21.262333,19.5324,"POLYGON ((59.78286 65.84086, 59.77802 65.79605..."
4,4_34,87513.0,141601.0,1247523,25947.0,69078.0,250000,7.014941,11.350572,10.3788,27.6312,18.365513,38.0100,"POLYGON ((-96.51418 62.72101, -96.51794 62.676..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,1_25,0.0,0.0,1248650,0.0,0.0,250000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,"POLYGON ((13.11395 11.47994, 13.11425 11.43474..."
127,4_23,0.0,0.0,1255514,0.0,0.0,250000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,"POLYGON ((114.16089 -4.17947, 114.16072 -4.224..."
128,4_3,15286.0,1016.0,1247625,15286.0,1016.0,250000,1.225208,0.081435,6.1144,0.4064,1.306643,6.5208,"POLYGON ((33.66383 -16.23972, 33.66398 -16.284..."
129,2_5,851.0,0.0,1250736,347.0,0.0,250000,0.068040,0.000000,0.1388,0.0000,0.068040,0.1388,"POLYGON ((30.73597 -26.90458, 30.73507 -26.949..."


In [8]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
images = gpd.read_file(image_table_data)
images

,collocated_dswx,dswx_cloud_cover,datetime,image_name,cloud_cover,site_coverage,instrument,provider,resolution,timeDelta_days,site_name,Strata,geometry
0,HLS.S30.T04WDB.2021271T225541.v2.0,4.0,2021-09-29T21:28:09,20210929_212809_16_2442,0.0,90.820013,PSB.SD,planetscope,3.0,0.938606,3_38,3.0,"POLYGON ((-160.00896 69.30478, -160.21043 69.1..."
1,HLS.S30.T33WXP.2021270T104719.v2.0,16.0,2021-09-27T09:55:37,20210927_095537_0f15,0.0,41.216128,PS2,planetscope,3.0,0.039303,4_43,4.0,"POLYGON ((17.22275 66.09223, 17.16459 66.02168..."
2,HLS.L30.T40WFU.2021268T073033.v2.0,2.0,2021-09-25T07:27:12,20210925_072712_16_2254,0.0,72.272530,PSB.SD,planetscope,3.0,0.002335,3_1,3.0,"POLYGON ((59.00446 65.95517, 58.85023 65.76646..."
3,HLS.S30.T14VPQ.2021244T175911.v2.0,2.0,2021-09-01T17:52:05,20210901_175205_71_2426,0.0,100.000000,PSB.SD,planetscope,3.0,0.010000,4_34,4.0,"POLYGON ((-96.94381 62.88876, -97.07609 62.707..."
4,HLS.S30.T52VFP.2021253T023549.v2.0,2.0,2021-09-11T00:51:29,20210911_005129_82_106a,0.0,100.000000,PS2.SD,planetscope,3.0,0.924389,3_30,3.0,"POLYGON ((130.86458 61.50211, 130.95358 61.357..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,HLS.S30.T14SQB.2021276T170141.v2.0,23.0,2021-10-03T16:16:39,20211003_161639_91_241d,0.0,100.000000,PSB.SD,planetscope,3.0,0.040347,3_40,3.0,"POLYGON ((-96.03320 33.15586, -96.08093 32.967..."
123,HLS.S30.T20LKN.2021283T143729.v2.0,14.0,2021-10-10T13:58:31,20211010_135831_84_227e,0.0,100.000000,PSB.SD,planetscope,3.0,0.032946,3_8,3.0,"POLYGON ((-64.96109 -11.14220, -64.99808 -11.3..."
124,HLS.S30.T35LPK.2021267T080651.v2.0,11.0,2021-09-24T08:20:25,20210924_082025_48_2424,0.0,100.000000,PSB.SD,planetscope,3.0,0.008675,4_37,4.0,"POLYGON ((28.20956 -9.14523, 28.17025 -9.34005..."
125,HLS.S30.T54KUG.2021267T005709.v2.0,3.0,2021-09-24T00:05:22,20210924_000522_94_2421,0.0,100.000000,PSB.SD,planetscope,3.0,0.039136,4_42,4.0,"POLYGON ((139.47770 -16.88868, 139.43688 -17.0..."


In [15]:
sites.head()

,site_name,water_pixels_orig,inundated_vegetation_orig,total_pixels_orig,water_pixels_crop,inundated_vegetation_crop,total_pixels_crop,percent_water_orig,percent_inundated_vegetation_orig,percent_water_crop,percent_inundated_vegetation_crop,percent_total_water_orig,percent_total_water_crop,geometry
0,3_38,0.0,0.0,1252157,0.0,0.0,250000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,"POLYGON ((-159.66522 69.26009, -159.66385 69.2..."
1,4_27,286582.0,8919.0,1251485,78861.0,3701.0,250000,22.899356,0.712673,31.5444,1.4804,23.612029,33.0248,"POLYGON ((-113.00031 67.04574, -112.99662 67.0..."
2,4_43,556111.0,41908.0,1244440,192790.0,2307.0,250000,44.687651,3.367619,77.1160,0.9228,48.055270,78.0388,"POLYGON ((17.81813 65.99817, 17.81319 65.95337..."
3,3_1,17647.0,246974.0,1244553,7271.0,41560.0,250000,1.417939,19.844394,2.9084,16.6240,21.262333,19.5324,"POLYGON ((59.78286 65.84086, 59.77802 65.79605..."
4,4_34,87513.0,141601.0,1247523,25947.0,69078.0,250000,7.014941,11.350572,10.3788,27.6312,18.365513,38.0100,"POLYGON ((-96.51418 62.72101, -96.51794 62.676..."


In [16]:
site_name = sites.iloc[2].site_name
sites[sites.site_name==site_name]

,site_name,water_pixels_orig,inundated_vegetation_orig,total_pixels_orig,water_pixels_crop,inundated_vegetation_crop,total_pixels_crop,percent_water_orig,percent_inundated_vegetation_orig,percent_water_crop,percent_inundated_vegetation_crop,percent_total_water_orig,percent_total_water_crop,geometry
2,4_43,556111.0,41908.0,1244440,192790.0,2307.0,250000,44.687651,3.367619,77.116,0.9228,48.05527,78.0388,"POLYGON ((17.81813 65.99817, 17.81319 65.95337..."


In [17]:
newSites = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry')
for index, site in sites.iterrows():
    xy = site.geometry.centroid.coords[0]
    UTM = utm.from_latlon(xy[1],xy[0])
    zone = UTM[2]
    letter = UTM[3]
    if letter >= "N":
        ns = 'north'
    else:
        ns = 'south'
    proj = '+proj=utm +zone='+str(zone)+' +'+ns+' +datum=WGS84 +units=m +no_defs'
    site_name = sites.iloc[index].site_name
    print(site_name)
    chip = sites[sites.site_name==site_name]
    print(chip.head())
    chip['geometry'] = chip.to_crs(proj).buffer(500,join_style=2).to_crs(epsg=4326)
    newSites = newSites.append(chip)
#for file in image_files:
#    if file[-3:] != 'tif':
#        continue
    
#    ID = file.split('_3B_Analytic')[0]
    #print(ID)
#    imagery = rasterio.open(download_dir+file)
#    image_crs = imagery.crs
#    chip_ID = images[images.image_name == ID].site_name.iloc[0]
#    chip = sites[sites.site_name == chip_ID]
#    chip['geometry'] = chip.geometry.to_crs(image_crs).buffer(-2500,join_style=2).to_crs(epsg=4326)
    
#    newSites = newSites.append(chip)

3_38
  site_name  water_pixels_orig  inundated_vegetation_orig  total_pixels_orig  \
0      3_38                0.0                        0.0            1252157   

   water_pixels_crop  inundated_vegetation_crop  total_pixels_crop  \
0                0.0                        0.0             250000   

   percent_water_orig  percent_inundated_vegetation_orig  percent_water_crop  \
0                 0.0                                0.0                 0.0   

   percent_inundated_vegetation_crop  percent_total_water_orig  \
0                                0.0                       0.0   

   percent_total_water_crop                                           geometry  
0                       0.0  POLYGON ((-159.66522 69.26009, -159.66385 69.2...  
4_27
  site_name  water_pixels_orig  inundated_vegetation_orig  total_pixels_orig  \
1      4_27           286582.0                     8919.0            1251485   

   water_pixels_crop  inundated_vegetation_crop  total_pixels_crop  \
1

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

2_24
  site_name  water_pixels_orig  inundated_vegetation_orig  total_pixels_orig  \
7      2_24               45.0                      824.0            1471092   

   water_pixels_crop  inundated_vegetation_crop  total_pixels_crop  \
7               26.0                      119.0             250000   

   percent_water_orig  percent_inundated_vegetation_orig  percent_water_crop  \
7            0.003059                           0.056013              0.0104   

   percent_inundated_vegetation_crop  percent_total_water_orig  \
7                             0.0476                  0.059072   

   percent_total_water_crop                                           geometry  
7                     0.058  POLYGON ((102.06018 59.90849, 102.06414 59.863...  
3_25
  site_name  water_pixels_orig  inundated_vegetation_orig  total_pixels_orig  \
8      3_25                0.0                        0.0            1249824   

   water_pixels_crop  inundated_vegetation_crop  total_pixels_crop  \
8

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

3_2
   site_name  water_pixels_orig  inundated_vegetation_orig  total_pixels_orig  \
14       3_2             9633.0                        0.0            1247246   

    water_pixels_crop  inundated_vegetation_crop  total_pixels_crop  \
14             5789.0                        0.0             250000   

    percent_water_orig  percent_inundated_vegetation_orig  percent_water_crop  \
14            0.772342                                0.0              2.3156   

    percent_inundated_vegetation_crop  percent_total_water_orig  \
14                                0.0                  0.772342   

    percent_total_water_crop  \
14                    2.3156   

                                             geometry  
14  POLYGON ((107.00194 57.15724, 106.99952 57.112...  
4_17
   site_name  water_pixels_orig  inundated_vegetation_orig  total_pixels_orig  \
15      4_17           334901.0                   100763.0            1248075   

    water_pixels_crop  inundated_vegetation_cro

...

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

3_9
    site_name  water_pixels_orig  inundated_vegetation_orig  \
121       3_9            16486.0                        0.0   

     total_pixels_orig  water_pixels_crop  inundated_vegetation_crop  \
121            1250172             8906.0                        0.0   

     total_pixels_crop  percent_water_orig  percent_inundated_vegetation_orig  \
121             250000            1.318699                                0.0   

     percent_water_crop  percent_inundated_vegetation_crop  \
121              3.5624                                0.0   

     percent_total_water_orig  percent_total_water_crop  \
121                  1.318699                    3.5624   

                                              geometry  
121  POLYGON ((113.88106 33.05787, 113.87959 33.012...  
4_31
    site_name  water_pixels_orig  inundated_vegetation_orig  \
122      4_31            97812.0                        0.0   

     total_pixels_orig  water_pixels_crop  inundated_vegetation_crop  \

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

4_3
    site_name  water_pixels_orig  inundated_vegetation_orig  \
128       4_3            15286.0                     1016.0   

     total_pixels_orig  water_pixels_crop  inundated_vegetation_crop  \
128            1247625            15286.0                     1016.0   

     total_pixels_crop  percent_water_orig  percent_inundated_vegetation_orig  \
128             250000            1.225208                           0.081435   

     percent_water_crop  percent_inundated_vegetation_crop  \
128              6.1144                             0.4064   

     percent_total_water_orig  percent_total_water_crop  \
128                  1.306643                    6.5208   

                                              geometry  
128  POLYGON ((33.66383 -16.23972, 33.66398 -16.284...  
2_5
    site_name  water_pixels_orig  inundated_vegetation_orig  \
129       2_5              851.0                        0.0   

     total_pixels_orig  water_pixels_crop  inundated_vegetation_crop  \


/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [27]:
newSites = sites
newSites.head()

,site_name,water_pixels_orig,inundated_vegetation_orig,total_pixels_orig,water_pixels_crop,inundated_vegetation_crop,total_pixels_crop,percent_water_orig,percent_inundated_vegetation_orig,percent_water_crop,percent_inundated_vegetation_crop,percent_total_water_orig,percent_total_water_crop,geometry
0,3_38,0.0,0.0,1252157,0.0,0.0,250000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,"POLYGON ((-159.66522 69.26009, -159.66385 69.2..."
1,4_27,286582.0,8919.0,1251485,78861.0,3701.0,250000,22.899356,0.712673,31.5444,1.4804,23.612029,33.0248,"POLYGON ((-113.00031 67.04574, -112.99662 67.0..."
2,4_43,556111.0,41908.0,1244440,192790.0,2307.0,250000,44.687651,3.367619,77.1160,0.9228,48.055270,78.0388,"POLYGON ((17.81813 65.99817, 17.81319 65.95337..."
3,3_1,17647.0,246974.0,1244553,7271.0,41560.0,250000,1.417939,19.844394,2.9084,16.6240,21.262333,19.5324,"POLYGON ((59.78286 65.84086, 59.77802 65.79605..."
4,4_34,87513.0,141601.0,1247523,25947.0,69078.0,250000,7.014941,11.350572,10.3788,27.6312,18.365513,38.0100,"POLYGON ((-96.51418 62.72101, -96.51794 62.676..."


In [25]:
newSites.to_file('newSites.geojson', driver="GeoJSON")  

In [28]:
newSites_bytes = bytes(newSites.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,'site.geojson')
s3object.put(Body=newSites_bytes)

{'ResponseMetadata': {'RequestId': 'NW4PEEH7XJ6XKK02',
  'HostId': 'STJZ7Krkbaz3khJOXojeSEX4q7FYCYFc/waBv1mjfbjpIk2ivsOAEKrMQBaeywvY48o+7zgYdi0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'STJZ7Krkbaz3khJOXojeSEX4q7FYCYFc/waBv1mjfbjpIk2ivsOAEKrMQBaeywvY48o+7zgYdi0=',
   'x-amz-request-id': 'NW4PEEH7XJ6XKK02',
   'date': 'Tue, 20 Sep 2022 22:48:44 GMT',
   'etag': '"9719e3db2887678234440fb2d1e22bcd"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9719e3db2887678234440fb2d1e22bcd"'}